## Report historical seasonality for the past or upcoming month

In [76]:
# If true looks at past 20 trading days,
#   if false looks at next 20 trading days

past = True

Import necessary packages

In [77]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

Calculate 20 day seasoanlity

In [78]:
# Read files from folder
folder = '../daily_prices'
files = os.listdir(folder)

# Create dataframe
seasonal_uplifts = dict()

for i, file in enumerate(files):
    # Read file
    df = pd.read_csv(os.path.join(folder, file))

    if not df.isna().any().any() and len(df.index) >= 504:
        # Decompose
        result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
        seasonal = result.seasonal

        # Impute outliers with mean
        z_scores = np.abs(stats.zscore(seasonal))
        outliers = z_scores > 3
        seasonal[outliers] = seasonal.mean()

        # normalize seasonality data
        mins, maxs = seasonal.min(), seasonal.max()
        seasonal = 2*(seasonal-mins)/(maxs-mins)-1

        # Get seasonality for next month (20 trading days)
        if past:
            seasonality = seasonal[-252:][-20:].reset_index(drop=True)
        else:
            seasonality = seasonal[-252:][:20].reset_index(drop=True)

        # Add to dataframe
        seasonal_uplifts[file[:-4]] = seasonality.to_list()

seasonal_uplifts = pd.DataFrame(seasonal_uplifts)

Report mean seasonality

In [79]:
# Calculate mean of each column
mean_values = seasonal_uplifts.mean()

# Create a new DataFrame with mean values and column names
avg_seasonal_uplifts = pd.DataFrame({
    'Stock': mean_values.index,
    '20_Day_Seasonality': mean_values.values
}).sort_values(by=['20_Day_Seasonality'], ascending=False).reset_index(drop=True)

avg_seasonal_uplifts

,Stock,20_Day_Seasonality
0,MOH,0.787226
1,CBOE,0.768031
2,NRG,0.762124
3,MPC,0.741053
4,ROL,0.725566
...,...,...
492,JNJ,-0.795714
493,NEM,-0.813004
494,WBD,-0.827643
495,SJM,-0.828307
